In [ ]:
import xml.etree.ElementTree as ET
from xml.dom import minidom
import datetime
from enum import Enum
from pydantic import BaseModel
from typing import List
from collections import defaultdict
from app.util import create_template_zip

class FileType(str, Enum):
    """파일 타입
    파일이 실행 파일인지, 프로젝트 파일인지, 소스코드인지 타입으로 분류하는 enum 클래스.
    """
    EXECUTION = "실행파일"
    CONF = "환경파일"
    DB = "DB파일"
    PROJECT = "프로젝트 파일"
    SOURCE = "소스코드"
    IMAGE = "이미지 파일"
    UNKNOWN = "기타 파일"

class FileData(BaseModel):
    """파일 데이터
    파일 정보를 구성하는 클래스
    """
    Device: str
    Csu: str
    Type: FileType
    Index: int
    FilePath: str
    Filename: str
    Version: str
    Size: int
    Checksum: str
    Date: str
    PartNumber: str
    Loc: str
    Description: str

data = [
    {"Device":"HDEV-001","Csu":"ADAgent","Type":FileType.CONF,"Index":1,"FilePath":"temp\\MC_ADAgent","Filename":".gitignore","Version":"1.0","Size":121,"Checksum":"11","Date":"2025-05-25","PartNumber":"Q45019224E","Loc":"","Description":""},
    {"Device":"HDEV-001","Csu":"ADAgent","Type":FileType.SOURCE,"Index":2,"FilePath":"temp\\MC_ADAgent","Filename":"anomalyagent","Version":"1.0","Size":8044,"Checksum":"11","Date":"2025-05-25","PartNumber":"Q45019224E","Loc":"33","Description":"이상 탐지 Anomalyagent 모듈"},
    {"Device":"HDEV-001","Csu":"ADAgent","Type":FileType.SOURCE,"Index":3,"FilePath":"temp\\MC_ADAgent","Filename":"inference","Version":"1.0","Size":7444,"Checksum":"11","Date":"2025-05-25","PartNumber":"Q45019224E","Loc":"17","Description":"Anomalyagent 패킷 데이터 추론 모듈"},
    {"Device":"HDEV-001","Csu":"ADAgent","Type":FileType.SOURCE,"Index":4,"FilePath":"temp\\MC_ADAgent","Filename":"internal_ad","Version":"1.0","Size":5457,"Checksum":"11","Date":"2025-05-25","PartNumber":"Q45019224E","Loc":"15","Description":"Anomalyagent 위협 신호 추론 모듈"},
    {"Device":"HDEV-001","Csu":"ADAgent","Type":FileType.EXECUTION,"Index":5,"FilePath":"temp\\MC_ADAgent","Filename":"mc_adagent","Version":"1.0","Size":329,"Checksum":"11","Date":"2025-05-25","PartNumber":"Q45019224E","Loc":"","Description":""},
    {"Device":"HDEV-001","Csu":"ADAgent","Type":FileType.CONF,"Index":6,"FilePath":"temp\\MC_ADAgent","Filename":"README","Version":"1.0","Size":625,"Checksum":"11","Date":"2025-05-25","PartNumber":"Q45019224E","Loc":"","Description":""},
    {"Device":"HDEV-001","Csu":"ADAgent","Type":FileType.EXECUTION,"Index":7,"FilePath":"temp\\MC_ADAgent","Filename":"requirements","Version":"1.0","Size":212,"Checksum":"11","Date":"2025-05-25","PartNumber":"Q45019224E","Loc":"","Description":""},
    {"Device":"HDEV-001","Csu":"ADAgent","Type":FileType.SOURCE,"Index":1,"FilePath":"temp\\MC_ADAgent\\app","Filename":"file_tailer","Version":"1.0","Size":12123,"Checksum":"11","Date":"2025-05-25","PartNumber":"Q45019224E","Loc":"102","Description":"파일 tailing 모듈"},
    {"Device":"HDEV-001","Csu":"ADAgent","Type":FileType.SOURCE,"Index":2,"FilePath":"temp\\MC_ADAgent\\app","Filename":"__init__","Version":"1.0","Size":542,"Checksum":"11","Date":"2025-05-25","PartNumber":"Q45019224E","Loc":"0","Description":"Logger 초기화"},
    {"Device":"HDEV-001","Csu":"ADAgent","Type":FileType.SOURCE,"Index":1,"FilePath":"temp\\MC_ADAgent\\app\\internal","Filename":"exception","Version":"1.0","Size":185,"Checksum":"11","Date":"2025-05-25","PartNumber":"Q45019224E","Loc":"2","Description":"예외 클래스 모듈"},
    {"Device":"HDEV-001","Csu":"ADAgent","Type":FileType.SOURCE,"Index":2,"FilePath":"temp\\MC_ADAgent\\app\\internal","Filename":"logger","Version":"1.0","Size":3096,"Checksum":"11","Date":"2025-05-25","PartNumber":"Q45019224E","Loc":"10","Description":"로거 등록 및 설정"},
    {"Device":"HDEV-001","Csu":"ADAgent","Type":FileType.PROJECT,"Index":1,"FilePath":"temp\\MC_ADAgent\\app\\model","Filename":"model","Version":"1.0","Size":5282,"Checksum":"11","Date":"2025-05-25","PartNumber":"Q45019224E","Loc":"47","Description":""},
    {"Device":"HDEV-001","Csu":"ADAgent","Type":FileType.PROJECT,"Index":2,"FilePath":"temp\\MC_ADAgent\\app\\model","Filename":"pca","Version":"1.0","Size":1699,"Checksum":"11","Date":"2025-05-25","PartNumber":"Q45019224E","Loc":"62","Description":""},
    {"Device":"HDEV-001","Csu":"ADAgent","Type":FileType.DB,"Index":3,"FilePath":"temp\\MC_ADAgent\\app\\model","Filename":"scaler","Version":"1.0","Size":892,"Checksum":"11","Date":"2025-05-25","PartNumber":"Q45019224E","Loc":"","Description":""},
    {"Device":"HDEV-001","Csu":"ADAgent","Type":FileType.DB,"Index":1,"FilePath":"temp\\MC_ADAgent\\app\\model\\gru","Filename":"model_gru_364","Version":"1.0","Size":98072,"Checksum":"11","Date":"2025-05-25","PartNumber":"Q45019224E","Loc":"","Description":""},
    {"Device":"HDEV-001","Csu":"ADAgent","Type":FileType.DB,"Index":2,"FilePath":"temp\\MC_ADAgent\\app\\model\\gru","Filename":"scaler_gru","Version":"1.0","Size":769,"Checksum":"11","Date":"2025-05-25","PartNumber":"Q45019224E","Loc":"","Description":""},
]

file_data_list: List[FileData] = [FileData(**item) for item in data]

exe_files = sorted(
    [file for file in file_data_list if file.Type in {FileType.EXECUTION, FileType.CONF, FileType.DB}],
    key=lambda x: x.FilePath
)

prj_files = sorted(
    [file for file in file_data_list if file.Type in {FileType.PROJECT}],
    key=lambda x: x.FilePath
)

src_files = sorted(
    [file for file in file_data_list if file.Type in {FileType.SOURCE, FileType.IMAGE}],
    key=lambda x: x.FilePath
)

unknown_files = sorted(
    [file for file in file_data_list if file.Type in {FileType.UNKNOWN}],
    key=lambda x: x.FilePath
)

exe_list = []
index = 0
path = ""
for file in exe_files:
    if path != file.FilePath:
        path = file.FilePath
        exe_list.append(['저장위치: ' + path])

    index += 1
    data = [
        file.Type.value,
        str(index),
        file.Filename,
        file.Version,
        str(file.Size),
        file.Checksum,
        file.Date,
        file.PartNumber + f"E{index:03d}",  # 숫자를 3자리 형식으로 포맷팅
        file.Description
    ]
    exe_list.append(data)

from app.hwpx import HWPXMLBuilder
builder = HWPXMLBuilder("./resources/section0.xml")
builder.add_para_number_text("실행파일", level=1)
builder.add_para_number_text("HDEV-001", level=2)
builder.add_text(f"  ○ HDEV-001의 실행파일 총 수 : {len(exe_files)}")
headers = ["구 분", "순번", "파일명", "버전", "크기 (Byte)", "첵섬", "수정일", "SW부품번호", "기능 설명"]
builder.add_table(exe_list, headers, "표", 1, "실행파일 목록")
builder.add_empty_paragraph()
builder.add_empty_paragraph()
builder.save('temp/11111/section0.xml')
create_template_zip("./temp/11111", "test.hwpx")


기존 폴더 삭제: ./temp/11111/template
'./resources/template' 내용을 './temp/11111/template'에 복사했습니다.
'./temp/11111/section0.xml'을 './temp/11111/template/Contents/section0.xml'에 복사했습니다.
'./temp/11111/template' 폴더가 'downloads/test.hwpx'으로 성공적으로 압축되었습니다.
